# DMTA Fractional Models: Glass Transition Fitting

Fractional viscoelastic models use power-law elements (springpots) to capture the broad relaxation spectra typical of glass transitions. With only 3-4 parameters, they achieve fits that require 10+ Prony terms in a Generalized Maxwell model.

## Learning Objectives
- Understand fractional order α as a measure of relaxation breadth
- Fit Fractional Zener Solid-Solid (FZSS) to DMTA data
- Compare fractional vs classical Zener fit quality
- Use `deformation_mode='tension'` for automatic E*→G* conversion

**Estimated Time:** 10 minutes

In [ ]:
import gc
import os
import sys
import warnings

import matplotlib.pyplot as plt
import numpy as np

if os.path.abspath(os.path.join(os.getcwd(), '../..')) not in sys.path:
    sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../..')))

from rheojax.core.jax_config import safe_import_jax

jax, jnp = safe_import_jax()

np.random.seed(42)
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11
warnings.filterwarnings('ignore', category=RuntimeWarning)

FAST_MODE = os.environ.get('FAST_MODE', '1') == '1'
print(f'FAST_MODE: {FAST_MODE}')

## 1. Synthetic DMTA Data with Broad Relaxation

Real polymers near Tg show a broad relaxation spectrum. We simulate this using the Cole-Davidson distribution, which produces an asymmetric, broad tan(δ) peak characteristic of glass transitions.

In [ ]:
# Generate broadened E*(ω) data using Cole-Davidson model
# G*(ω) = G_e + (G_g - G_e) * (1 - 1/(1 + iωτ)^β)
omega = np.logspace(-3, 4, 80 if FAST_MODE else 150)

# Material parameters (G-space)
G_e = 5e4     # Rubbery plateau (Pa)
G_g = 2e9     # Glassy plateau (Pa)
tau_cd = 0.1  # Central relaxation time (s)
beta_cd = 0.4 # Cole-Davidson exponent (broadening)
nu = 0.35     # Poisson's ratio (glassy polymer)
factor = 2 * (1 + nu)  # = 2.7

# Cole-Davidson G*(ω)
iw_tau = 1j * omega * tau_cd
G_star_cd = G_e + (G_g - G_e) * (1 - 1 / (1 + iw_tau)**beta_cd)

# Convert to E*
E_star_true = factor * G_star_cd

# Add noise (1%)
noise_r = 1 + 0.01 * np.random.randn(len(omega))
noise_i = 1 + 0.01 * np.random.randn(len(omega))
E_star = np.real(E_star_true) * noise_r + 1j * np.imag(E_star_true) * np.abs(noise_i)

print(f'E_rubbery = {G_e * factor:.2e} Pa')
print(f'E_glassy  = {G_g * factor:.2e} Pa')
print(f'E_g/E_r ratio = {G_g/G_e:.0f}')
print(f'Broadening β = {beta_cd}')
print(f'Poisson ratio ν = {nu} (glassy polymer)')

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.loglog(omega, np.real(E_star), 'ko', ms=3, alpha=0.6, label="E'")
ax1.loglog(omega, np.imag(E_star), 'rs', ms=3, alpha=0.6, label='E"')
ax1.set_xlabel('ω (rad/s)')
ax1.set_ylabel('Modulus (Pa)')
ax1.set_title('Synthetic DMTA Data (Broad Glass Transition)')
ax1.legend()

# tan(δ) = E''/E'
tan_delta = np.imag(E_star) / np.real(E_star)
ax2.semilogx(omega, tan_delta, 'g-', lw=2)
ax2.set_xlabel('ω (rad/s)')
ax2.set_ylabel('tan(δ)')
ax2.set_title('Loss Tangent (Broad Peak = Glass Transition)')
ax2.axhline(np.max(tan_delta), color='gray', ls='--', alpha=0.5)
ax2.text(omega[0]*2, np.max(tan_delta)*1.05, f'tan(δ)_max = {np.max(tan_delta):.3f}', fontsize=9)

plt.tight_layout()
plt.close('all')

## 2. Fractional Zener Solid-Solid (FZSS) Fit

The FZSS model uses a springpot (fractional dashpot) to capture broad relaxation:

$$G^*(\omega) = G_e + G_m \cdot \frac{(i\omega\tau_\alpha)^\alpha}{1 + (i\omega\tau_\alpha)^\alpha}$$

where α ∈ (0,1) controls the breadth of relaxation:
- α → 1: narrow (classical Maxwell/Zener)
- α → 0: very broad (stretched exponential-like)

In [ ]:
from rheojax.models.fractional.fractional_zener_ss import FractionalZenerSolidSolid

fzss = FractionalZenerSolidSolid()

# Fit E* data with automatic conversion
fzss.fit(
    omega, E_star,
    test_mode='oscillation',
    deformation_mode='tension',
    poisson_ratio=nu,
)

# Parameters are in G-space (model-native)
Ge_fit = fzss.parameters.get_value('Ge')
Gm_fit = fzss.parameters.get_value('Gm')
alpha_fit = fzss.parameters.get_value('alpha')
tau_fit = fzss.parameters.get_value('tau_alpha')

print('Fractional Zener SS Parameters (G-space):')
print(f'  Ge (equilibrium) = {Ge_fit:.4e} Pa  (true: {G_e:.4e})')
print(f'  Gm (Maxwell arm) = {Gm_fit:.4e} Pa  (true: {G_g - G_e:.4e})')
print(f'  alpha (frac order)= {alpha_fit:.4f}')
print(f'  tau_alpha         = {tau_fit:.4e} s')
print(f'\nα = {alpha_fit:.3f} → {"broad" if alpha_fit < 0.5 else "narrow"} relaxation spectrum')

## 3. Classical Zener Comparison

The classical Zener model (α = 1 fixed) has the same number of modulus parameters but cannot capture broad relaxation.

In [ ]:
from rheojax.models.classical.zener import Zener

zener = Zener()
zener.fit(
    omega, E_star,
    test_mode='oscillation',
    deformation_mode='tension',
    poisson_ratio=nu,
)

print('Classical Zener Parameters (G-space):')
for name in zener.parameters.keys():
    val = zener.parameters.get_value(name)
    print(f'  {name:10s} = {val:.4e}')

In [ ]:
E_fzss = fzss.predict(omega, test_mode='oscillation')
E_zener = zener.predict(omega, test_mode='oscillation')

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# E' comparison
ax = axes[0]
ax.loglog(omega, np.real(E_star), 'ko', ms=3, alpha=0.4, label='Data')
ax.loglog(omega, np.real(E_fzss), 'r-', lw=2, label=f'FZSS (α={alpha_fit:.2f})')
ax.loglog(omega, np.real(E_zener), 'b--', lw=2, label='Classical Zener')
ax.set_xlabel('ω (rad/s)')
ax.set_ylabel("E' (Pa)")
ax.set_title("Storage Modulus E'")
ax.legend(fontsize=9)

# E'' comparison
ax = axes[1]
ax.loglog(omega, np.imag(E_star), 'ko', ms=3, alpha=0.4, label='Data')
ax.loglog(omega, np.imag(E_fzss), 'r-', lw=2, label='FZSS')
ax.loglog(omega, np.imag(E_zener), 'b--', lw=2, label='Zener')
ax.set_xlabel('ω (rad/s)')
ax.set_ylabel('E" (Pa)')
ax.set_title('Loss Modulus E"')
ax.legend(fontsize=9)

# tan(δ) comparison
ax = axes[2]
tan_d_data = np.imag(E_star) / np.real(E_star)
tan_d_fzss = np.imag(E_fzss) / np.real(E_fzss)
tan_d_zener = np.imag(E_zener) / np.real(E_zener)
ax.semilogx(omega, tan_d_data, 'ko', ms=3, alpha=0.4, label='Data')
ax.semilogx(omega, tan_d_fzss, 'r-', lw=2, label='FZSS')
ax.semilogx(omega, tan_d_zener, 'b--', lw=2, label='Zener')
ax.set_xlabel('ω (rad/s)')
ax.set_ylabel('tan(δ)')
ax.set_title('Loss Tangent tan(δ)')
ax.legend(fontsize=9)

plt.tight_layout()
plt.close('all')

# R² comparison
for name, pred in [('FZSS', E_fzss), ('Zener', E_zener)]:
    residual = np.abs(E_star) - np.abs(pred)
    ss_res = np.sum(residual**2)
    ss_tot = np.sum((np.abs(E_star) - np.mean(np.abs(E_star)))**2)
    R2 = 1 - ss_res / ss_tot
    print(f'{name:8s}: R² = {R2:.6f}')

## 3b. Bayesian Uncertainty Quantification

NLSQ gives point estimates, but Bayesian inference via NumPyro NUTS provides:
- **Posterior distributions** for each parameter (credible intervals)
- **Correlation structure** between parameters (α vs τ_α trade-off)
- **Model adequacy diagnostics** (R-hat, ESS, divergences)

We use the FZSS model with NLSQ warm-start → NUTS sampling.

In [ ]:
# Bayesian inference on FZSS model
import arviz as az

# Re-create FZSS model for Bayesian (fresh instance)
fzss_bayes = FractionalZenerSolidSolid()
fzss_bayes.fit(
    omega, E_star,
    test_mode='oscillation',
    deformation_mode='tension',
    poisson_ratio=nu,
)

if FAST_MODE:
    num_warmup, num_samples = 50, 100
else:
    num_warmup, num_samples = 200, 500

try:
    bayes_result = fzss_bayes.fit_bayesian(
        omega, E_star,
        test_mode='oscillation',
        deformation_mode='tension',
        poisson_ratio=nu,
        num_warmup=num_warmup,
        num_samples=num_samples,
        num_chains=1,
        seed=42,
    )
    bayesian_completed = True
    print(f'NUTS sampling complete: {num_warmup} warmup + {num_samples} samples')

    # Summary statistics
    for name in ['Ge', 'Gm', 'alpha', 'tau_alpha']:
        if name in bayes_result.posterior_samples:
            samples = np.array(bayes_result.posterior_samples[name])
            print(f'  {name:12s}: {np.mean(samples):.4e} +/- {np.std(samples):.4e}')
except Exception as e:
    bayesian_completed = False
    print(f'Bayesian inference skipped: {e}')

In [ ]:
# ArviZ diagnostics: trace plot + forest plot
if bayesian_completed:
    idata = bayes_result.to_inference_data()

    # Filter degenerate parameters (range < 1e-6) for clean plots
    param_names = list(bayes_result.posterior_samples.keys())
    plot_params = []
    for name in param_names:
        if name.startswith('sigma') or name.startswith('_'):
            continue
        vals = np.array(bayes_result.posterior_samples[name])
        if np.ptp(vals) > 1e-10:
            plot_params.append(name)

    if plot_params and hasattr(idata, 'posterior'):
        fig_trace = az.plot_trace(idata, var_names=plot_params, figsize=(12, 3 * len(plot_params)))
        plt.tight_layout()
        plt.close('all')

        fig_forest = az.plot_forest(idata, var_names=plot_params, hdi_prob=0.95, figsize=(10, 4))
        plt.tight_layout()
        plt.close('all')

        print(f'Plotted {len(plot_params)} non-degenerate parameters: {plot_params}')
    else:
        print('No non-degenerate parameters to plot')
else:
    print('Skipped (Bayesian inference did not complete)')

In [ ]:
# Posterior predictive: credible intervals on E*(ω)
if bayesian_completed:
    omega_pred = np.logspace(-3, 4, 100)
    E_preds = []

    # Draw predictions from posterior samples
    n_draws = min(50, num_samples)
    indices = np.random.choice(num_samples, n_draws, replace=False)

    for idx in indices:
        # Create temporary model with posterior sample
        m_tmp = FractionalZenerSolidSolid()
        for name in ['Ge', 'Gm', 'alpha', 'tau_alpha']:
            if name in bayes_result.posterior_samples:
                val = float(np.array(bayes_result.posterior_samples[name]).flatten()[idx])
                m_tmp.parameters.set_value(name, val)
        m_tmp._test_mode = 'oscillation'
        m_tmp._deformation_mode = 'tension'
        m_tmp._poisson_ratio = nu
        try:
            pred = m_tmp.predict(omega_pred, test_mode='oscillation')
            E_preds.append(pred)
        except Exception:
            pass

    if E_preds:
        E_preds = np.array(E_preds)
        E_prime_preds = np.real(E_preds)
        E_dblp_preds = np.imag(E_preds)

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

        # E' with credible intervals
        median_Ep = np.median(E_prime_preds, axis=0)
        lo_Ep = np.percentile(E_prime_preds, 2.5, axis=0)
        hi_Ep = np.percentile(E_prime_preds, 97.5, axis=0)
        ax1.loglog(omega, np.real(E_star), 'ko', ms=3, alpha=0.4, label='Data')
        ax1.loglog(omega_pred, median_Ep, 'r-', lw=2, label='Median')
        ax1.fill_between(omega_pred, lo_Ep, hi_Ep, alpha=0.2, color='red', label='95% CI')
        ax1.set_xlabel('ω (rad/s)')
        ax1.set_ylabel("E' (Pa)")
        ax1.set_title("Posterior Predictive: E'")
        ax1.legend(fontsize=9)

        # E'' with credible intervals
        median_Epp = np.median(E_dblp_preds, axis=0)
        lo_Epp = np.percentile(E_dblp_preds, 2.5, axis=0)
        hi_Epp = np.percentile(E_dblp_preds, 97.5, axis=0)
        ax2.loglog(omega, np.imag(E_star), 'ko', ms=3, alpha=0.4, label='Data')
        ax2.loglog(omega_pred, median_Epp, 'b-', lw=2, label='Median')
        ax2.fill_between(omega_pred, lo_Epp, hi_Epp, alpha=0.2, color='blue', label='95% CI')
        ax2.set_xlabel('ω (rad/s)')
        ax2.set_ylabel('E" (Pa)')
        ax2.set_title('Posterior Predictive: E"')
        ax2.legend(fontsize=9)

        plt.tight_layout()
        plt.close('all')
        print(f'Posterior predictive from {len(E_preds)} draws')
    else:
        print('No valid posterior predictions generated')

    # Cleanup Bayesian model
    del fzss_bayes
    gc.collect()
    jax.clear_caches()
else:
    print('Skipped (Bayesian inference did not complete)')

## 4. Physical Interpretation

The fractional order α encodes the breadth of the relaxation time distribution:

| α value | Relaxation spectrum | Physical meaning |
|---------|-------------------|------------------|
| 0.9-1.0 | Narrow (single mode) | Simple polymer liquid |
| 0.5-0.8 | Moderate broadening | Typical amorphous polymer near Tg |
| 0.2-0.5 | Very broad | Highly heterogeneous dynamics |

For DMTA analysis, α < 0.5 suggests significant dynamic heterogeneity, common in:
- Amorphous polymers through the glass transition
- Polymer blends with partial miscibility
- Semi-crystalline polymers with amorphous/crystalline interphase

## 5. Fractional Models on Real Polymer Data

We now apply the same fractional and classical models to real DMTA data from the pyvisco project. The real polymer exhibits a broad glass transition — the kind of data where fractional models truly shine.

In [ ]:
import pandas as pd

# Load real frequency-domain master curve
data_dir = os.path.join(os.path.dirname(os.path.abspath('.')), 'dmta', 'data')
if not os.path.exists(data_dir):
    data_dir = os.path.join('.', 'data')

df_master = pd.read_csv(os.path.join(data_dir, 'freq_user_master.csv'), skiprows=[1])
omega_real = 2 * np.pi * df_master['f'].values
E_stor_real = df_master['E_stor'].values * 1e6  # MPa -> Pa
E_loss_real = df_master['E_loss'].values * 1e6
E_star_real = E_stor_real + 1j * E_loss_real

print(f'Real master curve: {len(omega_real)} points, {np.log10(omega_real.max()/omega_real.min()):.1f} decades')
print(f"E' range: {E_stor_real.min()/1e6:.0f} - {E_stor_real.max()/1e6:.0f} MPa")

In [ ]:
# Fit FZSS to real polymer data
fzss_real = FractionalZenerSolidSolid()
fzss_real.fit(
    omega_real, E_star_real,
    test_mode='oscillation',
    deformation_mode='tension',
    poisson_ratio=0.35,
)

alpha_real = fzss_real.parameters.get_value('alpha')
print(f'FZSS on real data:')
for name in fzss_real.parameters.keys():
    print(f'  {name:12s} = {fzss_real.parameters.get_value(name):.4e}')
print(f'\nalpha = {alpha_real:.3f} -> {"broad" if alpha_real < 0.5 else "moderate" if alpha_real < 0.8 else "narrow"} relaxation')

fzss_real_pred = fzss_real.predict(omega_real, test_mode='oscillation')

In [ ]:
# Fit classical Zener to same real data
zener_real = Zener()
zener_real.fit(
    omega_real, E_star_real,
    test_mode='oscillation',
    deformation_mode='tension',
    poisson_ratio=0.35,
)

zener_real_pred = zener_real.predict(omega_real, test_mode='oscillation')

print('Classical Zener on real data:')
for name in zener_real.parameters.keys():
    print(f'  {name:12s} = {zener_real.parameters.get_value(name):.4e}')

In [ ]:
# Compare FZSS vs Zener on real data
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# E' comparison
ax = axes[0]
ax.loglog(omega_real, E_stor_real, 'ko', ms=3, alpha=0.4, label='Real data')
ax.loglog(omega_real, np.real(fzss_real_pred), 'r-', lw=2, label=f'FZSS (alpha={alpha_real:.2f})')
ax.loglog(omega_real, np.real(zener_real_pred), 'b--', lw=2, label='Classical Zener')
ax.set_xlabel(chr(969) + ' (rad/s)')
ax.set_ylabel("E' (Pa)")
ax.set_title("E' comparison on real data")
ax.legend(fontsize=9)

# E'' comparison
ax = axes[1]
ax.loglog(omega_real, E_loss_real, 'ko', ms=3, alpha=0.4, label='Real data')
ax.loglog(omega_real, np.imag(fzss_real_pred), 'r-', lw=2, label='FZSS')
ax.loglog(omega_real, np.imag(zener_real_pred), 'b--', lw=2, label='Zener')
ax.set_xlabel(chr(969) + ' (rad/s)')
ax.set_ylabel('E" (Pa)')
ax.set_title('E" comparison on real data')
ax.legend(fontsize=9)

# tan(delta) comparison
ax = axes[2]
td_data = E_loss_real / E_stor_real
td_fzss = np.imag(fzss_real_pred) / np.maximum(np.real(fzss_real_pred), 1e-10)
td_zener = np.imag(zener_real_pred) / np.maximum(np.real(zener_real_pred), 1e-10)
ax.semilogx(omega_real, td_data, 'ko', ms=3, alpha=0.4, label='Real data')
ax.semilogx(omega_real, td_fzss, 'r-', lw=2, label='FZSS')
ax.semilogx(omega_real, td_zener, 'b--', lw=2, label='Zener')
ax.set_xlabel(chr(969) + ' (rad/s)')
ax.set_ylabel('tan(' + chr(948) + ')')
ax.set_title('Loss tangent on real data')
ax.legend(fontsize=9)

plt.tight_layout()
plt.close('all')

# R2 comparison
for name, pred in [('FZSS', fzss_real_pred), ('Zener', zener_real_pred)]:
    res_p = E_stor_real - np.real(pred)
    res_pp = E_loss_real - np.imag(pred)
    ss_res = np.sum(res_p**2) + np.sum(res_pp**2)
    ss_tot = np.sum((E_stor_real - np.mean(E_stor_real))**2) + np.sum((E_loss_real - np.mean(E_loss_real))**2)
    R2 = 1 - ss_res / ss_tot
    print(f'{name:8s}: R2 (combined) = {R2:.6f}')

print(f'\nSynthetic alpha = {alpha_fit:.3f}, Real alpha = {alpha_real:.3f}')

## 6. Bayesian UQ on Real DMTA Data

We apply the same NLSQ warm-start + NUTS workflow to the real polymer data. This quantifies parameter uncertainty and provides credible intervals on model predictions.

In [ ]:
# Bayesian inference on real data
fzss_bayes_real = FractionalZenerSolidSolid()
fzss_bayes_real.fit(
    omega_real, E_star_real,
    test_mode='oscillation',
    deformation_mode='tension',
    poisson_ratio=0.35,
)

if FAST_MODE:
    nw_real, ns_real = 50, 100
else:
    nw_real, ns_real = 200, 500

try:
    bayes_real = fzss_bayes_real.fit_bayesian(
        omega_real, E_star_real,
        test_mode='oscillation',
        deformation_mode='tension',
        poisson_ratio=0.35,
        num_warmup=nw_real,
        num_samples=ns_real,
        num_chains=1,
        seed=42,
    )
    bayes_real_ok = True
    print(f'NUTS on real data: {nw_real} warmup + {ns_real} samples')
    for name in ['Ge', 'Gm', 'alpha', 'tau_alpha']:
        if name in bayes_real.posterior_samples:
            s = np.array(bayes_real.posterior_samples[name])
            print(f'  {name:12s}: {np.mean(s):.4e} +/- {np.std(s):.4e}')
except Exception as e:
    bayes_real_ok = False
    print(f'Bayesian on real data failed: {e}')

In [ ]:
# Posterior predictive on real data with 95% credible intervals
if bayes_real_ok:
    omega_ppd = np.logspace(
        np.log10(omega_real.min()) - 0.5,
        np.log10(omega_real.max()) + 0.5,
        120,
    )
    E_ppd = []
    n_draws = min(50, ns_real)
    indices = np.random.choice(ns_real, n_draws, replace=False)

    for idx in indices:
        m_tmp = FractionalZenerSolidSolid()
        for name in ['Ge', 'Gm', 'alpha', 'tau_alpha']:
            if name in bayes_real.posterior_samples:
                val = float(np.array(bayes_real.posterior_samples[name]).flatten()[idx])
                m_tmp.parameters.set_value(name, val)
        m_tmp._test_mode = 'oscillation'
        m_tmp._deformation_mode = 'tension'
        m_tmp._poisson_ratio = 0.35
        try:
            pred = m_tmp.predict(omega_ppd, test_mode='oscillation')
            E_ppd.append(pred)
        except Exception:
            pass

    if E_ppd:
        E_ppd = np.array(E_ppd)
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

        # E' with CI
        med = np.median(np.real(E_ppd), axis=0)
        lo = np.percentile(np.real(E_ppd), 2.5, axis=0)
        hi = np.percentile(np.real(E_ppd), 97.5, axis=0)
        ax1.loglog(omega_real, E_stor_real, 'ko', ms=3, alpha=0.4, label='Real data')
        ax1.loglog(omega_ppd, med, 'r-', lw=2, label='Median')
        ax1.fill_between(omega_ppd, lo, hi, alpha=0.2, color='red', label='95% CI')
        ax1.set_xlabel(chr(969) + ' (rad/s)')
        ax1.set_ylabel("E' (Pa)")
        ax1.set_title("Posterior predictive: E' (real data)")
        ax1.legend()

        # E'' with CI
        med = np.median(np.imag(E_ppd), axis=0)
        lo = np.percentile(np.imag(E_ppd), 2.5, axis=0)
        hi = np.percentile(np.imag(E_ppd), 97.5, axis=0)
        ax2.loglog(omega_real, E_loss_real, 'ko', ms=3, alpha=0.4, label='Real data')
        ax2.loglog(omega_ppd, med, 'b-', lw=2, label='Median')
        ax2.fill_between(omega_ppd, lo, hi, alpha=0.2, color='blue', label='95% CI')
        ax2.set_xlabel(chr(969) + ' (rad/s)')
        ax2.set_ylabel('E" (Pa)')
        ax2.set_title('Posterior predictive: E" (real data)')
        ax2.legend()

        plt.tight_layout()
        plt.close('all')
        print(f'Posterior predictive from {len(E_ppd)} draws on real data')

    # ArviZ diagnostics
    idata_real = bayes_real.to_inference_data()
    plot_params_real = []
    for name in bayes_real.posterior_samples.keys():
        if name.startswith('sigma') or name.startswith('_'):
            continue
        vals = np.array(bayes_real.posterior_samples[name])
        if np.ptp(vals) > 1e-10:
            plot_params_real.append(name)

    if plot_params_real and hasattr(idata_real, 'posterior'):
        fig_trace = az.plot_trace(idata_real, var_names=plot_params_real, figsize=(12, 3 * len(plot_params_real)))
        plt.tight_layout()
        plt.close('all')

    del fzss_bayes_real
    gc.collect()
    jax.clear_caches()
else:
    print('Skipped (Bayesian on real data did not complete)')

## Key Takeaways

- **Fractional Zener (FZSS)** captures broad DMTA transitions with only 4 parameters
- **alpha parameter** directly measures relaxation breadth (smaller alpha = broader transition)
- **Classical Zener fails** for broad glass transitions (too narrow tan(delta) peak)
- **Real polymer data** confirms the advantage of fractional models for broad spectra
- **Bayesian UQ** quantifies parameter uncertainty and provides prediction credible intervals
- **`deformation_mode='tension'`** handles E*->G* conversion automatically

## Next Steps

- `04_dmta_relaxation.ipynb`: Time-domain relaxation fitting
- `06_dmta_model_selection.ipynb`: Systematic model comparison

In [ ]:
del fzss, zener
gc.collect()
jax.clear_caches()
print('Cleanup complete')